In [1]:
import pandas as pd
import numpy as np
import os
import json

In [5]:
from elasticsearch import Elasticsearch, RequestsHttpConnection
host = 'https://search-lmi-capstone-2-525zkk33t4z5iy6ozqd63ctgmq.us-east-1.es.amazonaws.com'
es = Elasticsearch(host)
print(es.info())

{'name': 'eb7b3348d3c34ce7dc22f263930c51ad', 'cluster_name': '846033058400:lmi-capstone-2', 'cluster_uuid': 'QzTffcNgStmet053IRSP2w', 'version': {'number': '7.9.1', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-11-03T09:54:32.349659Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


In [6]:
os.getcwd()

'C:\\Users\\kaleb\\Documents\\GitHub\\kaleb_LMI_local\\LMI-Capstone-1\\2018'

In [7]:
text_file = open('CMS-2018-0101-0001.txt', 'r')

In [8]:
content = text_file.read()

In [140]:
# if ('feel' in content):
#     print("word foud")
# else:
#     print("word not found")

In [9]:
# unique_comments = pd.read_json('unique_comments2018.json')
fjson = open('unique_comments2018.json')
unique_comments = json.load(fjson)

In [10]:
keys = unique_comments.keys()
values = unique_comments.values()

In [11]:
def comment_upload(comments_dict, es_index):
    for key, vals in comments_dict.items():
            values_dict = {'text':vals}
            res = es.index(index=es_index, id=key, body=values_dict, doc_type='_doc')
            es.indices.refresh(index=es_index)
    print("Last id uploaded:", key)

In [16]:
comment_upload(unique_comments,'comment_index')

Last id uploaded: 0467


In [12]:
termVectorBody = {
  "fields" : ["text"],
  "term_statistics" : True,
  "field_statistics" : True,
  "offsets" : False,
  "payloads" : False,
  "positions" : False,
    "filter": {
    "max_num_terms": 20,
    "min_term_freq": 1,
    "min_doc_freq": 1,
    "max_doc_freq": 5
  }
}

In [13]:
termvecs = dict()
for section in keys:
    result = es.termvectors(index = "comment_index", id = section, body= termVectorBody)
    if (result['found']):
        termvecs[section] = list(result['term_vectors']['text']['terms'].keys())

In [14]:
# words = pd.DataFrame(data=termvecs).T
words = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in termvecs.items() ]))

<ipython-input-14-3d8ea2919584>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  words = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in termvecs.items() ]))


In [15]:
words

,0002,0003,0004,0005,0006,0007,0008,0009,0010,0011,...,0399,0401,0404,0408,0419,0423,0428,0437,0438,0467
0,accord,absolut,aco',allianc,clarif,100,0938,compani,cannot,100,...,10,1034,ceic4,amount,crisi,80,16th,210,baptist,0001
1,alik,aco',advis,attain,my,aledad,"1,100",complex,reserv,alter,...,2017,2333,cific,bill,cross,acoâ,1993,67756,disagre,0101
2,broom,antagon,becom,bend,NaN,allevi,414,cultur,worth,am,...,2019,82.8,communitv,coast,cut,deliv,3832,attn,foot,107
3,catalyst.nejm.org,bend,feel,blame,NaN,cardiologist,42,doubt,NaN,cash,...,30,98557,defin,diabet,diseas,disast,384,biggest,give,206.216.2541
4,constantli,bring,impract,curv,NaN,certif,425,enforc,NaN,graduat,...,31,cah,f,equip,disord,field,41863,cheyenn,NaN,445
5,diminish,burnout,qp,cut,NaN,contain,administ,game,NaN,isn't,...,7,care.â,fohc,exam,futur,learn,alfero,counti,NaN,45
6,dynam,chanc,regular,death,NaN,count,aggress,highli,NaN,kind,...,9,clallam,ny,ey,mix,natur,andrew,elast,NaN,along
7,entic,concept,septemb,effectu,NaN,destroi,at45,larger,NaN,margin,...,adequ,counti,tk,florida,opioid,next,cah,embrac,NaN,associationâ
8,forc,drg,taken,fear,NaN,fqhc,behind,left,NaN,reserv,...,administ,district,www.chcanys.org,foot,particularli,now,chair,franci,NaN,assumpt
9,hold,duti,timelin,government,NaN,home,cfr,misguid,NaN,starter,...,collabor,harbor,york,invok,substanc,opioid,charli,frontier,NaN,barton


In [71]:
nrule = []
for x in list(termvecs.values()):
    for y in x:
        if (y in content):
            pass
        else:
            nrule.append(y)

In [74]:
len(nrule)

596

In [73]:
nrule = list(set(nrule))

In [47]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaleb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
from nltk.corpus import wordnet 

In [ ]:
# for i in nrule:
#     for sys in wordnet.synsets(i):
#         for l in sys.lemmas():
#             print(l.name())
#         print("-----------------------------------{}-----------------------------------".format(i))

In [49]:
synonym_dict = {}
for i in nrule:
    synonym_list = []
    for sys in wordnet.synsets(i):
        for l in sys.lemmas():
            synonym_list.append(l.name())
    if(len(synonym_list) !=0):
        synonym_dict[i] = list(set(synonym_list)) 

In [26]:
li = ['the', 'man', 'in', 'the', 'iron', 'mask']
rule = ",".join(li) + ' => '+'the target'+ '\n'
print(rule)

the,man,in,the,iron,mask => the target



In [75]:
rule = ''
for i in nrule:
    synonym_list = []
    for sys in wordnet.synsets(i):
        for l in sys.lemmas():
            synonym_list.append(l.name())
    if(len(synonym_list) !=0):
        synonym_d = list(set(synonym_list))
        rule = rule + ', '.join(synonym_d) + ' => ' + i + '\n'
fi = open('newsyn.txt', 'w')
fi.write(rule)
fi.close()

In [53]:
rule

'president, chair, chairwoman, chairperson, chairman => chairman\nAugusta, capital_of_Maine => augusta\nSouth, southward, due_south, S, Confederacy, south, Confederate_States_of_America, Confederate_States, in_the_south, to_the_south, Dixieland, Dixie => south\nKY, Bluegrass_State, Kentucky => ky\nflyer, broadside, orbitual, broadsheet, throwaway, round, handbill, rotary, flier, circular, bill => circular\nRichmond, capital_of_Virginia => richmond\npsychiatrist, shrink, head-shrinker => psychiatrist\nAmerican_language, American, American_English => american\nfinger, palpate, spirit, flavour, flavor, experience, find, sense, tone, tactile_property, look, feel, feeling, smell => feel\nUnion, northerly, north, magnetic_north, Second_Earl_of_Guilford, compass_north, N, Frederick_North, northwards, northward, due_north, North => north\nmemorandum, memo, memoranda => memorandum\nreinvent => reinvent\nstandard, banner, streamer => banner\nsmall_fry, minor, youngster, tyke, tiddler, tike, nest

In [19]:
from PyDictionary import PyDictionary
dictionary=PyDictionary()
answer = dictionary.synonym('kill')

In [21]:
# type(answer)

# the use case of the comment problem statement and architecture of the comment 
# the architecture of the momemntts.
#





In [23]:
# if (type(answer) == list):
#     print('the answer is: ', answer)

In [ ]:
count = 0
for i in nrule:
    answer = dictionary.synonym(i,True)
    count +=1
    if (type(answer) == list):
        count +=1
#         print(answer)
#         print('This is shte numner you are looking for', count)
    if (count > 10):
        break;
        